In [53]:
import pandas as pd

In [ ]:
#"/Users/JaWook/Desktop/UCI-VIRT-DATA-PT-12-2024-U-LOLC-MWTH/02-Homework/18-Tableau/Data/ File paths for local usage
file_1 = 202402-citibike-tripdata.csv"
file_2 = "/Users/JaWook/Desktop/UCI-VIRT-DATA-PT-12-2024-U-LOLC-MWTH/02-Homework/18-Tableau/Data/202407-citibike-tripdata_1.csv"


In [55]:
# Load datasets
df1 = pd.read_csv(file_1)
df2 = pd.read_csv(file_2)

/var/folders/j5/k10__xzx4vg4137jr9wwfprm0000gp/T/ipykernel_25855/4292383543.py:2: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv(file_1)
/var/folders/j5/k10__xzx4vg4137jr9wwfprm0000gp/T/ipykernel_25855/4292383543.py:3: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv(file_2)


In [56]:
# Clean df2: drop Unnamed columns and fill missing station info
df2 = df2.loc[:, ~df2.columns.str.contains("^Unnamed")]
df2['end_station_id'] = df2['end_station_id'].fillna('unknown')
df2['end_station_name'] = df2['end_station_name'].fillna('unknown')


In [57]:
# Standardize and clean both datasets
def clean_and_prepare(df):
    df = df.rename(columns={
        'start_station_id': 'start_station_id',
        'start_station_name': 'start_name',
        'start_lat': 'start_latitude',
        'start_lng': 'start_longitude',
        'end_station_id': 'end_station_id',
        'end_station_name': 'end_name',
        'end_lat': 'end_latitude',
        'end_lng': 'end_longitude'
    })
    df['started_at'] = pd.to_datetime(df['started_at'], errors='coerce')
    df['ended_at'] = pd.to_datetime(df['ended_at'], errors='coerce')
    df['trip_length'] = (df['ended_at'] - df['started_at']).dt.total_seconds() / 60
    df = df[df['trip_length'] >= 1]
    df['start_day'] = df['started_at'].dt.day_name()
    df['end_day'] = df['ended_at'].dt.day_name()
    for col in ['start_latitude', 'start_longitude', 'end_latitude', 'end_longitude']:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    return df

df1_clean = clean_and_prepare(df1)
df2_clean = clean_and_prepare(df2)


In [58]:
# Sample 250 from each and combine
df1_sample = df1_clean.sample(n=250, random_state=1)
df2_sample = df2_clean.sample(n=250, random_state=2)
combined_df = pd.concat([df1_sample, df2_sample], ignore_index=True)


In [61]:
# Export to CSV
output_path = "/Users/JaWook/Desktop/UCI-VIRT-DATA-PT-12-2024-U-LOLC-MWTH/02-Homework/18-Tableau/Data/cleaned_combined_citibike.csv"
combined_df.to_csv(output_path, index=False)

In [62]:
# Optional: print preview
print("✅ Combined dataset saved to:", output_path)
print(combined_df.head())


✅ Combined dataset saved to: /Users/JaWook/Desktop/UCI-VIRT-DATA-PT-12-2024-U-LOLC-MWTH/02-Homework/18-Tableau/Data/cleaned_combined_citibike.csv
            ride_id  rideable_type              started_at  \
0  C75F0BC5707A87B0   classic_bike 2024-02-02 19:18:33.830   
1  8108C3D4E2AFD5BD  electric_bike 2024-02-09 18:09:24.268   
2  9094D2A3710985F4   classic_bike 2024-02-16 17:46:02.744   
3  D254BB55D9197151  electric_bike 2024-02-11 17:54:56.244   
4  B7B41EC37D6C6879  electric_bike 2024-02-02 20:43:38.670   

                 ended_at                  start_name start_station_id  \
0 2024-02-02 19:25:14.768  Bergen St & Vanderbilt Ave          4157.10   
1 2024-02-09 18:12:09.967             1 Ave & E 94 St          7286.05   
2 2024-02-16 17:51:26.530             E 15 St & 3 Ave          5863.07   
3 2024-02-11 18:08:31.419             E 12 St & Ave C          5616.08   
4 2024-02-02 20:48:20.143              E 1 St & 1 Ave          5593.01   

                     end_name end_st